In [1]:
import tensorflow as tf 
import numpy as np 
import pandas as pd
from TFData.Dataset import Dataset
from model.models import ModelCreation
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model

In [2]:
IMAGE_SIZE = [1024,1024]
IMAGE_SIZE_DESIRED = [256,256]
DATASET_SIZE = 33127
BATCH_SIZE = 8
EPOCHS = 5 

In [3]:
training_dataset = tf.io.gfile.glob('./Dataset/tfrecords/train*.tfrec')
test_dataset = tf.io.gfile.glob('./Dataset/tfrecords/test*.tfrec')

In [4]:
DataGenerator = Dataset(train_files=training_dataset,
                        test_files=test_dataset,
                        validation_split=0.2,
                        image_size=IMAGE_SIZE,
                        dataset_size=DATASET_SIZE,
                        batch_size=BATCH_SIZE,
                        resize_shape=IMAGE_SIZE_DESIRED)

In [5]:
# Here if you want to create training and validation dataset you must invoke get_train_and_validation_dataset
# ... at least I hope so 
DataGenerator.get_train_and_validation_dataset()

In [6]:
model_util = ModelCreation(architecture='efficientnet',
                      learning_rate= 0.00075,
                      input_shape=(*IMAGE_SIZE_DESIRED,3),
                      output_shape=1,
                      linear=True,
                      verbose=True,
                      loss = binary_crossentropy,
                      optimizer = Adam,
                      metric = binary_crossentropy)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
efficientnet-b5 (Model)      (None, 8, 8, 2048)        28513520  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 2048)              4196352   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 2049      
Total params: 32,711,921
Trainable params: 32,539,185
Non-trainable params: 172,736
_________________________________________________________________
Created model: efficientnet
Input Shape: (256, 256, 3)
Output Shape: 1
Mixed precission is set


In [15]:
callbacks = [
    ModelCheckpoint('my_model.h5',monitor='val_loss',verbose=1,save_best_only=True),
    EarlyStopping(patience=2)
]

In [16]:
history = model_util.model.fit_generator(
                                DataGenerator.train_tfdataset,
                                steps_per_epoch = DataGenerator.get_train_steps_per_epoch(),
                                epochs = 5,
                                validation_data = DataGenerator.validation_tfdataset,
                                validation_steps = DataGenerator.get_validation_steps_per_epoch(),
                                verbose = 1,
                                callbacks=model_util.inject_callbacks(callbacks))

W0608 17:20:13.690399 16032 deprecation.py:323] From <ipython-input-16-5e53364b4b2e>:8: Model.fit_generator (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/5
3313/3313 [==============================] - ETA: 0s - loss: 0.1015 - binary_crossentropy: 0.1018
Epoch 00001: val_loss improved from inf to 0.07945, saving model to my_model.h5

Epoch 00001: val_loss did not improve from 0.07945
3313/3313 [==============================] - 4939s 1s/step - loss: 0.1015 - binary_crossentropy: 0.1018 - val_loss: 0.0795 - val_binary_crossentropy: 0.0795
Epoch 2/5
3313/3313 [==============================] - ETA: 0s - loss: 0.0890 - binary_crossentropy: 0.0890
Epoch 00002: val_loss improved from 0.07945 to 0.07785, saving model to my_model.h5

Epoch 00002: val_loss did not improve from 0.07785
3313/3313 [==============================] - 4932s 1s/step - loss: 0.0890

In [8]:
test_dataset = DataGenerator.get_test_dataset()
test_images = test_dataset.map(lambda image,idnum:tf.image.resize(image,[256,256]))

In [9]:
Loaded_model = load_model('my_model.h5')

In [10]:
probabilites = Loaded_model.predict(test_images)

In [12]:
NUM_TEST_IMAGES = 10982
test_ids_ds = test_dataset.map(lambda image,idnum: idnum).unbatch()
test_ids = next(iter(test_ids_ds.batch(NUM_TEST_IMAGES))).numpy().astype('U')

In [13]:
sub = pd.read_csv('./Dataset/sample_submission.csv')
pred_df = pd.DataFrame({'image_name': test_ids,'target': np.concatenate(probabilites)})
del sub['target']
sub = sub.merge(pred_df,on='image_name')

In [16]:
sub.to_csv("EfficientNetB5Sub.csv",index=False)